### Import library

In [1]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image
import time

### Download training dataset

In [2]:
# Download training dataset
ROOT = '/content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data'
train_data = datasets.MNIST(
    root=ROOT,
    train=True,
    download=True
)
test_data = datasets.MNIST(
    root=ROOT,
    train=False,
    download=True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 56.7MB/s]


Extracting /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 2.83MB/s]

Extracting /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 13.3MB/s]


Extracting /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.48MB/s]


Extracting /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/data/MNIST/raw



### Split training dataset

In [3]:

# Splitting training: validation 0.9: 0.1
VALID_RATIO = 0.9
n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(
    train_data,
    [n_train_examples, n_valid_examples]
)

# compute mean and std for normalization
mean = train_data.dataset.data.float().mean() / 255
std = train_data.dataset.data.float().std() / 256

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])

train_data.dataset.transform = train_transforms
valid_data.dataset.transform = test_transforms

# Create dataloader
BATCH_SIZE = 256
train_dataloader = data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=BATCH_SIZE
)

valid_dataloader = data.DataLoader(
    valid_data,
    batch_size=BATCH_SIZE
)

### Define model

In [4]:

class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=6, kernel_size=5, padding='same'
        )
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(16*5*5, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc_1(outputs)
        outputs = self.fc_2(outputs)
        outputs = self.fc_3(outputs)
        return outputs

In [5]:

def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0

    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(inputs)

        # compute loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| Epoch {:3d} | {:5d}/{:5d} batches"
                "| Accuracy {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc/total_count
                ))

            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

# Evaluation function


def evaluate(model, criterion, valid_dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)
            loss = criterion(predictions, labels)
            losses.append(loss)

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)

    return epoch_acc, epoch_loss

### Train model

In [9]:

# Training
num_classes = len(train_data.dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lenet_model = LeNetClassifier(num_classes)
lenet_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet_model.parameters())

num_epochs = 10
save_model = '/content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_1/model'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100
for epoch in range(1, num_epochs + 1):
    epoch_start_time = time.time()

    # training
    train_acc, train_loss = train(
        lenet_model, optimizer, criterion, train_dataloader, device, epoch)

    train_accs.append(train_acc)
    train_losses.append(train_loss)

    # evaluate
    eval_acc, eval_loss = evaluate(lenet_model, criterion, valid_dataloader)
    eval_accs.append(eval_acc)
    eval_losses.append(eval_loss)

    # save best model
    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model.pt')

    # print loss, acc and epoch
    print("===========")
    print(
        "| End of epoch {:3d} | time: {:5.2f}s | train accuracy {:8.3f} | train loss {:8.3f}"
        "| Valid Accuracy {:8.3f} | Valid loss {:8.3f}".format(epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss))
    print("===========")

    # load best model
    lenet_model.load_state_dict(torch.load(save_model + '/lenet_model.pt'))
    lenet_model.eval()

| Epoch   1 |    50/  211 batches| Accuracy    0.686
| Epoch   1 |   100/  211 batches| Accuracy    0.890
| Epoch   1 |   150/  211 batches| Accuracy    0.929
| Epoch   1 |   200/  211 batches| Accuracy    0.943
| End of epoch   1 | time: 29.51s | train accuracy    0.943 | train loss    0.462| Valid Accuracy    0.947 | Valid loss    0.181


<ipython-input-9-224e602874d9>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet_model.load_state_dict(torch.load(save_model + '/lenet_model.pt'))


| Epoch   2 |    50/  211 batches| Accuracy    0.960
| Epoch   2 |   100/  211 batches| Accuracy    0.966
| Epoch   2 |   150/  211 batches| Accuracy    0.965
| Epoch   2 |   200/  211 batches| Accuracy    0.967
| End of epoch   2 | time: 29.47s | train accuracy    0.971 | train loss    0.118| Valid Accuracy    0.972 | Valid loss    0.100
| Epoch   3 |    50/  211 batches| Accuracy    0.974
| Epoch   3 |   100/  211 batches| Accuracy    0.975
| Epoch   3 |   150/  211 batches| Accuracy    0.972
| Epoch   3 |   200/  211 batches| Accuracy    0.974
| End of epoch   3 | time: 29.43s | train accuracy    0.974 | train loss    0.086| Valid Accuracy    0.976 | Valid loss    0.085
| Epoch   4 |    50/  211 batches| Accuracy    0.975
| Epoch   4 |   100/  211 batches| Accuracy    0.977
| Epoch   4 |   150/  211 batches| Accuracy    0.980
| Epoch   4 |   200/  211 batches| Accuracy    0.978
| End of epoch   4 | time: 29.04s | train accuracy    0.975 | train loss    0.074| Valid Accuracy    0.977

### Evaluate model

In [10]:
# Evaluation model
test_data.transform = test_transforms
test_dataloader = data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE
)

test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader)
test_acc, test_loss

(0.988, tensor(0.0353))